In [ ]:
import os

REPO = "SolarFlareZ/aml-project"
BRANCH = "main"
PROJECT_DIR = "/content/aml-project"

if os.path.exists(PROJECT_DIR):
    print("repo exists, pulling last commit")
    %cd {PROJECT_DIR}
    !git checkout {BRANCH}
    !git pull origin {BRANCH}
else:
    print("cloning repo...")
    !git clone -b {BRANCH} https://github.com/{REPO}.git
    %cd {PROJECT_DIR}

!pip install -q -r requirements.txt

print(f"\ncurr dir: {os.getcwd()}")

In [ ]:
!python -m src.train_fedavg \
    federated.num_clients=10 \
    federated.num_rounds=5 \
    federated.participation_rate=0.5 \
    logging.eval_every=1 \
    experiment_name=fedavg_quick_test

In [ ]:
# IID
!python -m src.train_fedavg \
    federated.num_clients=100 \
    federated.num_rounds=500 \
    federated.participation_rate=0.1 \
    federated.local_steps=4 \
    federated.sharding=iid \
    logging.eval_every=10 \
    experiment_name=fedavg_iid

In [ ]:
# non IID Nc = 5
for J in [4, 8, 16]:
    print(f"\n{'='*60}")
    print(f"Running: Nc=5, J={J}")
    print(f"{'='*60}")
    !python -m src.train_fedavg \
        federated.num_clients=100 \
        federated.num_rounds=500 \
        federated.participation_rate=0.1 \
        federated.local_steps={J} \
        federated.sharding=shard \
        federated.num_classes_per_client=5 \
        logging.eval_every=10 \
        experiment_name=fedavg_nc5_j{J}

In [ ]:
# non IID Nc = 1
for J in [4, 8, 16]:
    print(f"\n{'='*60}")
    print(f"Running: Nc=1, J={J}")
    print(f"{'='*60}")
    !python -m src.train_fedavg \
        federated.num_clients=100 \
        federated.num_rounds=500 \
        federated.participation_rate=0.1 \
        federated.local_steps={J} \
        federated.sharding=shard \
        federated.num_classes_per_client=1 \
        logging.eval_every=10 \
        experiment_name=fedavg_nc1_j{J}